In [1]:
import os
import subprocess
from pyarrow import fs
import pyarrow as pa

classpath = subprocess.Popen(["/home/hadoop/hadoop/bin/hdfs", "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
os.environ["CLASSPATH"] = classpath.decode("utf-8")
hdfs = fs.HadoopFileSystem(host='192.168.0.160', port=8020, user='hadoop')


2024-06-26 00:20:30,885 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 디렉토리 경로
dir_path = '/'

# 디렉토리 목록 가져오기
file_infos = hdfs.get_file_info(pa.fs.FileSelector(dir_path, recursive=False))  
for file_info in file_infos:
    print(file_info.path) 



/1111111
/20240619
/20280619
/2chung1
/awannagohome
/bobb
/bsh
/chy
/class5-0617
/class5_0617
/cma
/cmh
/cts
/dob
/dragonair
/encore
/gohome
/hazy
/hungry
/jeong
/jeong_0614
/jotaesik
/kdg
/kdh
/khj
/kkh
/kkh5
/kkk
/kse
/ksj
/leshen
/lhe
/lllIlll
/ma0619
/mama
/mort
/mort_0617
/mort_20240619
/mydata
/naverNews
/psw
/rsh
/scw
/ssy
/stock
/test2
/tmp
/user
/wannagohome
/yjy
/yjy_test.parquet
/ys
/yssw


In [12]:
from pyspark.sql import SparkSession
spark.stop()
spark = SparkSession.builder.appName("www").getOrCreate()

df = spark.read.csv("hdfs:////encore/tpss_bcycl_od_statnhm_202001.csv", encoding='cp949', header=True)


In [39]:
spark

In [10]:
data = [("James", "Sales", 3000), \
    ("Michael", "Sales", 4600), \
    ("Robert", "Sales", 4100), \
    ("Maria", "Finance", 3000), \
    ("James", "Sales", 3000), \
    ("Scott", "Finance", 3300), \
    ("Jen", "Finance", 3900), \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000), \
    ("Saif", "Sales", 4100) \
  ]

In [11]:
# Create DataFrame
columns= ["employee_name", "department", "salary"]
test = spark.createDataFrame(data = data, schema = columns)


In [14]:
df.select("시작_대여소ID").distinct().count()

1543

In [15]:
df.first()

Row(기준_날짜='20200101', 구분코드='0', 기준_시간='0000', 시작_대여소ID='ST-443', 시작_대여소명='성산2동_041_2', 종료_대여소ID='ST-82', 종료_대여소명='성산2동_041_3', 전체건수='1', 전체이용시간(분)='2', 전체이용거리(m)='450')

In [16]:
df.printSchema()

root
 |-- 기준_날짜: string (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: string (nullable = true)
 |-- 전체이용시간(분): string (nullable = true)
 |-- 전체이용거리(m): string (nullable = true)



In [21]:
# 형변환
from pyspark.sql.functions import  col
df2 = df.withColumn("전체이용거리(m)", col("전체이용거리(m)").cast('int'))

In [22]:
from pyspark.sql.types import IntegerType
df3 = df.withColumn("전체 이용거리(m)", col("전체이용거리(m)").cast(IntegerType()))

In [24]:
df3.

+---------+--------+---------+-------------+----------------+-------------+----------------+--------+----------------+---------------+----------------+
|기준_날짜|구분코드|기준_시간|시작_대여소ID|   시작_대여소명|종료_대여소ID|   종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|전체 이용거리(m)|
+---------+--------+---------+-------------+----------------+-------------+----------------+--------+----------------+---------------+----------------+
| 20200101|       0|     0000|       ST-443|   성산2동_041_2|        ST-82|   성산2동_041_3|       1|               2|            450|             450|
| 20200101|       0|     0000|        ST-56|   여의동_005_10|      ST-1321|  영등포동_036_1|       1|              10|           1460|            1460|
| 20200101|       0|     0000|      ST-1546|    조원동_004_1|       ST-703|    신사동_023_1|       1|               5|            700|             700|
| 20200101|       1|     0000|      ST-1701|   당산2동_065_1|      ST-1701|   당산2동_065_1|       1|              28|          70670|           70670|
| 20200101|       

In [25]:
from pyspark.sql.functions import mean
df2.select(mean(col("전체이용거리(m)"))).show()

+--------------------+
|avg(전체이용거리(m))|
+--------------------+
|  3970.4106343727362|
+--------------------+



In [26]:
df2.withColumn("전체이용시간(분)", col("전체이용시간(분)").cast('int')).select(mean(col("전체이용시간(분)"))).show()

+---------------------+
|avg(전체이용시간(분))|
+---------------------+
|   19.249510036830372|
+---------------------+



In [31]:
from pyspark.sql.functions import StructType
schema = StructType().\
      add('기준_날짜',  StringType(), True ).\
        add('구분코드',  StringType(), True ).\
        add('기준_시간',  StringType(), True ).\
        add('시작_대여소ID',  StringType(), True ).\
        add('시작_대여소명',  StringType(), True ).\
        add('종료_대여소ID',  StringType(), True ).\
        add('종료_대여소명',  StringType(), True ).\
        add('전체건수',  IntegerType(), True ).\
        add('전체이용시간(분)',  IntegerType(), True ).\
        add('전체이용거리(m)' , IntegerType(), True )

In [32]:
tmp = spark.read.format("csv").\
        option("header", True).\
        option("encoding", 'cp949').\
        schema(schema).\
        load("hdfs:///encore/tpss_bcycl_od_statnhm_202001.csv")


In [33]:
tmp.printSchema()

root
 |-- 기준_날짜: string (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: integer (nullable = true)
 |-- 전체이용시간(분): integer (nullable = true)
 |-- 전체이용거리(m): integer (nullable = true)



In [ ]:
from pyspark.sql.functions import mean, col, max, min, stddev

tmp.select( stddev('전체이용시간(분)')).show()


tmp.select( mean('전체이용시간(분)')).show()


In [ ]:
from pyspark.sql.functions import coalesce
tmp.select(coalesce('전체이용시간(분)')).count()

In [ ]:
# Returns the first column that is not null.
cDf = spark.createDataFrame([(None, None), (1, None), (None, 2)], ("a", "b"))
cDf.show()

In [40]:
cDf = spark.createDataFrame([(None, None, None), (None, 1, None), (None, 2, 6), (6, 3, 4)], ("a", "b", "c"))

In [41]:
cDf.show()

+----+----+----+
|   a|   b|   c|
+----+----+----+
|NULL|NULL|NULL|
|NULL|   1|NULL|
|NULL|   2|   6|
|   6|   3|   4|
+----+----+----+



In [43]:
# isnull
from pyspark.sql.functions import isnull
cDf.select(isnull("a")).show()

+-----------+
|(a IS NULL)|
+-----------+
|       true|
|       true|
|       true|
|      false|
+-----------+



In [44]:
cDf.filter(isnull("a") | isnull("b")).count()

3

In [45]:
from pyspark.sql.functions import col, when, count
cDf.agg(*[count(when(isnull(x), x)).\
          alias("null") 
             for x in cDf.columns]).show()

+----+----+----+
|null|null|null|
+----+----+----+
|   3|   1|   2|
+----+----+----+



In [46]:
when_test = spark.createDataFrame([("A", 10), ("B", 20), ("C", 30)], ["category", "value"])

df_new = when_test.withColumn("new_col", when(col("value") > 20, "high").otherwise("low"))
df_new.show()

+--------+-----+-------+
|category|value|new_col|
+--------+-----+-------+
|       A|   10|    low|
|       B|   20|    low|
|       C|   30|   high|
+--------+-----+-------+

